## Packages

In [1]:
install.packages("car")
install.packages("dplyr")
install.packages("haven")
install.packages("lfe")
install.packages("sandwich")
install.packages("plm")
install.packages("stargazer")
install.packages("corrplot")
install.packages("ivreg")
install.packages("gbm")
install.packages("fixest")

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Warning message in install.packages("dplyr"):
“installation of package ‘dplyr’ had non-zero exit status”
Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Warning message in install.packages("haven"):
“installation of package ‘haven’ had non-zero exit status”
Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Warning message in install.packages("lfe"):
“installation of package ‘lfe’ had non-zero exit status”
Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)



In [1]:
options(max.print = 20)

In [2]:
library(haven)
library(lfe)
library(lmtest)
library(dplyr)
library(stargazer)
library(sandwich)
library(corrplot)
library(ivreg)
library(purrr)
library(stringr)
library(tidyr)
library(xtable)
library(lfe)
library(mgcv)
library(gbm)
library(fixest)
library(reshape2)
library(ivreg)

Loading required package: Matrix

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘lmtest’


The following object is masked from ‘package:lfe’:

    waldtest



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


corrplot 0.92 loaded


Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack


Loading required package: nlme


Attaching package: ‘nlme’


The following object is masked from ‘package:dplyr’:

    collapse


This is mgcv 1.8-39. For overview type 'help("mgcv-package")'.

Loaded

In [3]:
set.seed(123)
star_points = c(0, 0.01, 0.05, 0.1, 1)
star_symbols = c("***", "**", "*", " ")

wrap_with_makecell <- function(x) {
  paste0("\\makecell{", x, "}")
}

print_latex <- function (table_data) {
  # table_data <- lapply(table_data, function(x) lapply(x, wrap_with_makecell))
  # table_data <- data.frame(table_data, check.names = FALSE)
  table_data <- xtable(table_data)
  print(table_data, sanitize.text.function = function(x) gsub("\n", "\\\\\\\\", x))
}

In [4]:
extract_2_coefs_factory <- function(feature1, feature2) {
  extract_coefs_fn <- function(model) {
    summary_model <- summary(model)
    coefs <- summary_model$coefficients

    # Find the rows for experiment_treatment_commute120 and experiment_treatment
    treatment_row <- which(row.names(coefs) == feature1)
    announcement_experiment_row <- which(row.names(coefs) == feature2)

    # Extract coefficients and standard errors
    data.frame(
      feature = c(feature1, feature2),
      values = c(
        if (length(treatment_row) > 0) {
          stars <- symnum(coefs[treatment_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
          str_c(round(coefs[treatment_row, 1], digits = 4), stars, "\n(", round(coefs[treatment_row, 2], digits = 4), ")") 
        } else "",
        if (length(announcement_experiment_row) > 0) {
          stars <- symnum(coefs[announcement_experiment_row, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
          str_c(round(coefs[announcement_experiment_row, 1], digits = 4), stars, "\n(", round(coefs[announcement_experiment_row, 2], digits = 4), ")")
        } else ""
      )
    )
  }

  extract_coefs_fn
}

In [5]:
extract_first_regressor <- function(model) {
  summary_model <- summary(model)
  coefs <- summary_model$coefficients
  
  non_intercept_rows <- row.names(coefs) != "(Intercept)"
  first_non_intercept <- which(non_intercept_rows)[1]
  
  coef <- coefs[first_non_intercept, 1]
  se <- coefs[first_non_intercept, 2]
  stars <- symnum(coefs[first_non_intercept, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
  
  regressor <- row.names(coefs)[first_non_intercept]
  
  data.frame(regressor = regressor,
             values = str_c(round(coef, digits=4), stars, "\n(", round(se, digits=4), ")"))
}

## Table 2

In [6]:
performance_during_exper <- haven::read_dta("data/performance_during_exper.dta")

col_1_regression <-  felm(perform1 ~ experiment_treatment + as.factor(year_week)
                          | personid | 0 | personid, 
                          data=performance_during_exper, 
                          subset=(year_week != 201049 & (expgroup == 1 | expgroup == 0)))

summary(col_1_regression)

col_2_regression <- felm(perform1 ~ treatment + as.factor(year_week) | 0 | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week > 201049 & (expgroup == 1 | expgroup == 0)))

summary(col_2_regression)

col_3_regression <- felm(phonecall ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logcallpersec) & phonecallraw > 20))

summary(col_3_regression)

col_4_regression <- felm(logphonecall ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logcallpersec) & phonecallraw > 20))

summary(col_4_regression)

col_5_regression <- felm(logcallpersec ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

summary(col_5_regression)

col_6_regression <- felm(logcalllength ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

summary(col_6_regression)

# wage_data <- read.dta("wage_new.dta")
# col_7_regression <- felm(loggrosswage ~ experiment_treatment + as.factor(wage_month) | personid | 0 | personid,
#                          data = wage_data,
#                          subset = (wage_month < 201109))

# extract_first_regressor <- extract_2_coefs_factory("experiment_treatment", "treatment")

model_list <- list(col_1_regression, col_2_regression, col_3_regression, col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list %>%
  map_dfr(extract_first_regressor, .id = "model")

table_data <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

table_data
print_latex(table_data)


Call:
   felm(formula = perform1 ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0))) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.2689 -0.4187  0.0424  0.4687  3.7219 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.23236      0.06235   3.727  0.00024 ***
as.factor(year_week)201002  1.62942      0.08519  19.127  < 2e-16 ***
as.factor(year_week)201003  1.76952      0.08865  19.961  < 2e-16 ***
as.factor(year_week)201004  1.87433      0.09033  20.750  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7826 on 17472 degrees of freedom
  (705 observations deleted due to missingness)
Multiple R-squared(full model): 0.4018   Adjusted R-squared: 0.3904 
Multiple R-squa


Call:
   felm(formula = perform1 ~ treatment + as.factor(year_week) |      0 | 0 | personid, data = performance_during_exper, subset = (year_week >      201049 & (expgroup == 1 | expgroup == 0))) 

Residuals:
    Min      1Q  Median      3Q     Max 
-3.3496 -0.6955 -0.0188  0.6251  5.1145 

Coefficients:
                            Estimate Cluster s.e. t value Pr(>|t|)    
(Intercept)                -0.342084     0.070767  -4.834 1.37e-06 ***
treatment                   0.183651     0.085310   2.153 0.031372 *  
as.factor(year_week)201051  0.131411     0.056614   2.321 0.020305 *  
as.factor(year_week)201052  0.054956     0.064242   0.855 0.392331    
 [ reached getOption("max.print") -- omitted 34 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.9708 on 7426 degrees of freedom
  (26 observations deleted due to missingness)
Multiple R-squared(full model): 0.06676   Adjusted R-squared: 0.06211 
Multiple R-squared(proj model): 0.0667


Call:
   felm(formula = phonecall ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0) & !is.na(logcallpersec) & phonecallraw > 20)) 

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6044 -0.3321  0.0625  0.4346  2.8137 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.24769      0.05773    4.29 3.41e-05 ***
as.factor(year_week)201002  2.01296      0.09859   20.42  < 2e-16 ***
as.factor(year_week)201003  2.16757      0.10579   20.49  < 2e-16 ***
as.factor(year_week)201004  2.37959      0.09994   23.81  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6954 on 9207 degrees of freedom
  (17 observations deleted due to missingness)
Multiple R-squared(full model): 0.4122   


Call:
   felm(formula = logphonecall ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0) & !is.na(logcallpersec) & phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.82994 -0.08958  0.05860  0.18380  1.03639 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.11983      0.02479   4.833 3.65e-06 ***
as.factor(year_week)201002  1.13626      0.07239  15.697  < 2e-16 ***
as.factor(year_week)201003  1.20426      0.06585  18.288  < 2e-16 ***
as.factor(year_week)201004  1.27837      0.06370  20.069  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.34 on 9207 degrees of freedom
  (17 observations deleted due to missingness)
Multiple R-squared(full model)


Call:
   felm(formula = logcallpersec ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0) & !is.na(logphonecall) & phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.47497 -0.04937  0.00081  0.04991  1.47859 

Coefficients:
                             Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        3.172e-02    1.289e-02   2.461 0.015133 *  
as.factor(year_week)201002 -3.272e-02    1.307e-02  -2.503 0.013534 *  
as.factor(year_week)201003 -2.845e-02    1.174e-02  -2.424 0.016677 *  
as.factor(year_week)201004 -2.218e-02    1.285e-02  -1.726 0.086705 .  
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.084 on 9207 degrees of freedom
  (12 observations deleted due to missingness)
Multiple R-squared(


Call:
   felm(formula = logcalllength ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0) & !is.na(logphonecall) & phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.91905 -0.09002  0.05829  0.18720  0.92845 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.08811      0.02684   3.283  0.00131 ** 
as.factor(year_week)201002  1.16897      0.07078  16.515  < 2e-16 ***
as.factor(year_week)201003  1.23271      0.06422  19.194  < 2e-16 ***
as.factor(year_week)201004  1.30056      0.06221  20.906  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3378 on 9207 degrees of freedom
  (12 observations deleted due to missingness)
Multiple R-squared(full mode

regressor,1,2,3,4,5,6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
experiment_treatment,0.2324*** (0.0624),NA,0.2477*** (0.0577),0.1198*** (0.0248),0.0317** (0.0129),0.0881*** (0.0268)
treatment,NA,0.1837** (0.0853),NA,NA,NA,NA


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Fri May 10 03:33:53 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllllll}
  \hline
 & regressor & 1 & 2 & 3 & 4 & 5 & 6 \\ 
  \hline
1 & experiment_treatment & 0.2324***\\(0.0624) &  & 0.2477***\\(0.0577) & 0.1198***\\(0.0248) & 0.0317**\\(0.0129) & 0.0881***\\(0.0268) \\ 
  2 & treatment &  & 0.1837**\\(0.0853) &  &  &  &  \\ 
   \hline
\end{tabular}
\end{table}


In [7]:
# col_1_data = performance_during_exper[(performance_during_exper$year_week != 201049 & (performance_during_exper$expgroup == 1 | performance_during_exper$expgroup == 0) & !is.na(performance_during_exper$perform1) & !is.na(performance_during_exper$experiment_treatment) & !is.na(performance_during_exper$year_week)), ]
# corrs = list()
# sum = 0
# for (unique_person_id in unique(col_1_data$personid)) {
#   idx = which(col_1_data$personid == unique_person_id)
#   cor = cov(col_1_regression$residuals[idx, ], col_1_data$experiment_treatment[idx])
#   sum = sum + abs(cor)
#   corrs[length(corrs) + 1] = abs(cor)
# }
# sum / length(corrs)

# feols(perform1 ~ experiment_treatment + as.factor(year_week) | personid,
#                    data = performance_during_exper,
#                    subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0)),
#                    cluster = "personid")

In [8]:
idx <- sample(1 : nrow(performance_during_exper), nrow(performance_during_exper) * 0.8)
performance_during_exper_train <- performance_during_exper[idx, ]
performance_during_exper_test <- performance_during_exper[-idx, ]

felm_model <- felm(perform1 ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                   data = performance_during_exper_train,
                   subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0)))

gam_model <- gam(perform1 ~ experiment_treatment + as.factor(year_week) + as.factor(personid),
                 data = performance_during_exper_train,
                 subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0)))

# AIC
felm_aic <- AIC(felm_model)
gam_aic <- gam_model$aic
cat("felm AIC:", felm_aic, "\ngam AIC:", gam_aic, "\n")

# performance_during_exper_train_pred <- performance_during_exper_train[, names(performance_during_exper_train) != "personid"]
# performance_during_exper_test_pred <- performance_during_exper_test[, names(performance_during_exper_test) != "personid"]

# # prediction
# felm_pred_train <- model.matrix(felm_model$formula, data = performance_during_exper_train_pred) %*% felm_model$coefficients
# gam_pred_train <- predict(gam_model, newdata = performance_during_exper_train_pred)
# felm_pred_test <- model.matrix(felm_model$formula, data = performance_during_exper_test_pred) %*% felm_model$coefficients
# gam_pred_test <- predict(gam_model, newdata = performance_during_exper_test_pred)

# felm_loss_train <- mean((performance_during_exper_train$perform1 - felm_pred_train)^2)
# gam_loss_train <- mean((performance_during_exper_train$perform1 - gam_pred_train)^2)
# felm_loss_test <- mean((performance_during_exper_train$perform1 - felm_pred_test)^2)
# gam_loss_test <- mean((performance_during_exper_train$perform1 - gam_pred_test)^2)
# cat("felm train loss:", felm_loss_train)
# cat("gam train loss:", gam_loss_train)
# cat("felm test loss:", felm_loss_test)
# cat("gam test loss:", gam_loss_test)

felm AIC: 33663.04 
gam AIC: 33663.04 


## Table 3

In [9]:
col_1_regression <- felm(logcalllength ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

summary(col_1_regression)

col_2_regression <- felm(logcall_dayworked ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

summary(col_2_regression)

col_3_regression <- felm(logdaysworked ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20 & !is.na(logcall_dayworked)))

summary(col_3_regression)

col_4_regression <- felm(logcalllength ~ experiment_treatment_commute120 + experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

summary(col_4_regression)

col_5_regression <- felm(logcall_dayworked ~ experiment_treatment_commute120 + experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20))

summary(col_5_regression)

col_6_regression <- felm(logdaysworked ~ experiment_treatment_commute120 + experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = performance_during_exper,
                         subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) & phonecallraw > 20 & !is.na(logcall_dayworked)))

summary(col_6_regression)


extract_treatment120_coefs <- extract_2_coefs_factory("experiment_treatment", "experiment_treatment_commute120")

model_list <- list(col_1_regression, col_2_regression, col_3_regression, col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list %>%
  map_dfr(extract_treatment120_coefs, .id = "model")

table_data <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

table_data
print_latex(table_data)


Call:
   felm(formula = logcalllength ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0) & !is.na(logphonecall) & phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.91905 -0.09002  0.05829  0.18720  0.92845 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.08811      0.02684   3.283  0.00131 ** 
as.factor(year_week)201002  1.16897      0.07078  16.515  < 2e-16 ***
as.factor(year_week)201003  1.23271      0.06422  19.194  < 2e-16 ***
as.factor(year_week)201004  1.30056      0.06221  20.906  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3378 on 9207 degrees of freedom
  (12 observations deleted due to missingness)
Multiple R-squared(full mode


Call:
   felm(formula = logcall_dayworked ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0) & !is.na(logphonecall) & phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.71283 -0.10257  0.04145  0.16063  0.78719 

Coefficients:
                            Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.063424     0.023953   2.648 0.009080 ** 
as.factor(year_week)201002  0.186763     0.045867   4.072 7.96e-05 ***
as.factor(year_week)201003  0.231836     0.045374   5.109 1.10e-06 ***
as.factor(year_week)201004  0.239893     0.042823   5.602 1.17e-07 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.279 on 9207 degrees of freedom
  (12 observations deleted due to missingness)
Multiple R-squared(f


Call:
   felm(formula = logdaysworked ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = performance_during_exper,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0) & !is.na(logphonecall) & phonecallraw > 20 & !is.na(logcall_dayworked))) 

Residuals:
     Min       1Q   Median       3Q      Max 
-1.77949 -0.06634  0.01799  0.12080  0.65052 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.02468      0.01227   2.012   0.0463 *  
as.factor(year_week)201002  0.98221      0.05183  18.950   <2e-16 ***
as.factor(year_week)201003  1.00087      0.04448  22.500   <2e-16 ***
as.factor(year_week)201004  1.06066      0.04511  23.511   <2e-16 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2017 on 9207 degrees of freedom
Multiple R-squared(full model): 0.3206   Adjust


Call:
   felm(formula = logcalllength ~ experiment_treatment_commute120 +      experiment_treatment + as.factor(year_week) | personid |      0 | personid, data = performance_during_exper, subset = (year_week !=      201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) &      phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.91956 -0.08988  0.05810  0.18597  0.92035 

Coefficients:
                                Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment_commute120  0.06935      0.03622   1.915   0.0577 .  
experiment_treatment             0.06938      0.03008   2.307   0.0226 *  
as.factor(year_week)201002       1.16929      0.07077  16.523   <2e-16 ***
as.factor(year_week)201003       1.23310      0.06419  19.211   <2e-16 ***
 [ reached getOption("max.print") -- omitted 82 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3376 on 9206 degrees of freedom
  (12 observati


Call:
   felm(formula = logcall_dayworked ~ experiment_treatment_commute120 +      experiment_treatment + as.factor(year_week) | personid |      0 | personid, data = performance_during_exper, subset = (year_week !=      201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) &      phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.70710 -0.10325  0.04176  0.16033  0.78077 

Coefficients:
                                 Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment_commute120  0.054908     0.031097   1.766 0.079740 .  
experiment_treatment             0.048598     0.027249   1.783 0.076792 .  
as.factor(year_week)201002       0.187009     0.045850   4.079 7.76e-05 ***
as.factor(year_week)201003       0.232141     0.045351   5.119 1.05e-06 ***
 [ reached getOption("max.print") -- omitted 82 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2789 on 9206 degrees of freedom
  (12 


Call:
   felm(formula = logdaysworked ~ experiment_treatment_commute120 +      experiment_treatment + as.factor(year_week) | personid |      0 | personid, data = performance_during_exper, subset = (year_week !=      201049 & (expgroup == 1 | expgroup == 0) & !is.na(logphonecall) &      phonecallraw > 20 & !is.na(logcall_dayworked))) 

Residuals:
     Min       1Q   Median       3Q      Max 
-1.78111 -0.06612  0.01818  0.12030  0.65070 

Coefficients:
                                Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment_commute120  0.01444      0.01672   0.864    0.389    
experiment_treatment             0.02079      0.01272   1.634    0.105    
as.factor(year_week)201002       0.98228      0.05184  18.950   <2e-16 ***
as.factor(year_week)201003       1.00095      0.04448  22.503   <2e-16 ***
 [ reached getOption("max.print") -- omitted 82 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2017 on 9206 degree

feature,1,2,3,4,5,6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
experiment_treatment,0.0881*** (0.0268),0.0634*** (0.024),0.0247** (0.0123),0.0694** (0.0301),0.0486* (0.0272),0.0208 (0.0127)
experiment_treatment_commute120,,,,0.0693* (0.0362),0.0549* (0.0311),0.0144 (0.0167)


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Fri May 10 03:34:00 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllllll}
  \hline
 & feature & 1 & 2 & 3 & 4 & 5 & 6 \\ 
  \hline
1 & experiment_treatment & 0.0881***\\(0.0268) & 0.0634***\\(0.024) & 0.0247**\\(0.0123) & 0.0694**\\(0.0301) & 0.0486*\\(0.0272) & 0.0208 \\(0.0127) \\ 
  2 & experiment_treatment_commute120 &  &  &  & 0.0693*\\(0.0362) & 0.0549*\\(0.0311) & 0.0144 \\(0.0167) \\ 
   \hline
\end{tabular}
\end{table}


## Table 4

In [10]:
# performance_during_exper <- haven::read_dta("data/performance_during_exper.dta")

# model_perform1_excluding_2 <- felm(
#   formula = perform1 ~ experiment_treatment + experiment_control + as.factor(year_week) |
#   personid | 0 | personid,
#   subset = (year_week != 201049 & expgroup != 2),
#   data = performance_during_exper
# )

# model_phonecall_excluding_2 <- felm(
#   formula = phonecall ~ experiment_treatment + experiment_control + factor(year_week) |
#   personid | 0 | personid,
#   subset = (year_week != 201049 & expgroup != 2),
#   data = performance_during_exper
# )

# model_perform1_excluding_3 <- felm(
#   formula = perform1 ~ experiment_treatment + experiment_control + factor(year_week) |
#   personid | 0 | personid,
#   subset = (year_week != 201049 & expgroup != 3),
#   data = performance_during_exper
# )

# model_phonecall_excluding_3 <- felm(
#   formula = phonecall ~ experiment_treatment + experiment_control + factor(year_week) |
#   personid | 0 | personid,
#   subset = (year_week != 201049 & expgroup != 3),
#   data = performance_during_exper
# )

# summary(model_perform1_excluding_2)

# extract_quality_coefs <- extract_2_coefs_factory("experiment_treatment", "experiment_control")

# model_list <- list(model_perform1_excluding_2, model_phonecall_excluding_2)

# model_data <- model_list %>%
#   map_dfr(extract_quality_coefs, .id = "model")

# table_data <- model_data %>%
#   pivot_wider(names_from = model, values_from = values)

# table_data
# print_latex(table_data)

## Table 5

In [11]:
selection_data <- read_dta("data/selection_new.dta")

col_1_regression <- felm(perform1 ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & year_week <= 201133))

summary(col_1_regression)

col_2_regression <- felm(perform1 ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049))

summary(col_2_regression)

col_3_regression <- felm(perform1 ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & balanced == 1))

summary(col_3_regression)

col_4_regression <- felm(logphonecall ~ experiment_treatment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & year_week <= 201133 & phonecallraw > 20))

summary(col_4_regression)

col_5_regression <- felm(logphonecall ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & phonecallraw > 20))

summary(col_5_regression)

col_6_regression <- felm(logphonecall ~ experiment_treatment + treatment_post_experiment + as.factor(year_week) | personid | 0 | personid,
                         data = selection_data,
                         subset = (year_week != 201049 & phonecallraw > 20 & balanced == 1))

summary(col_6_regression)

extract_selection_coefs = extract_2_coefs_factory("experiment_treatment", "treatment_post_experiment")

model_list <- list(col_1_regression, col_2_regression, col_3_regression, col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list %>%
  map_dfr(extract_selection_coefs, .id = "model")

table_data <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

table_data
print_latex(table_data)


Call:
   felm(formula = perform1 ~ experiment_treatment + as.factor(year_week) |      personid | 0 | personid, data = selection_data, subset = (year_week !=      201049 & year_week <= 201133)) 

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6269 -0.4207  0.0312  0.4494  3.7859 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.24423      0.05819   4.197 3.77e-05 ***
as.factor(year_week)201002  1.63337      0.08088  20.194  < 2e-16 ***
as.factor(year_week)201003  1.76537      0.08400  21.016  < 2e-16 ***
as.factor(year_week)201004  1.87319      0.08456  22.151  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 81 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7376 on 17281 degrees of freedom
  (1007 observations deleted due to missingness)
Multiple R-squared(full model): 0.4367   Adjusted R-squared: 0.4258 
Multiple R-squared(proj model): 0.1354   Adju

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”



Call:
   felm(formula = perform1 ~ experiment_treatment + treatment_post_experiment +      as.factor(year_week) | personid | 0 | personid, data = selection_data,      subset = (year_week != 201049)) 

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5383 -0.4501  0.0233  0.4837  4.0609 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.22118      0.04845   4.565 7.88e-06 ***
treatment_post_experiment   0.28387      0.08138   3.488 0.000575 ***
as.factor(year_week)201002  1.62277      0.08101  20.031  < 2e-16 ***
as.factor(year_week)201003  1.75812      0.08400  20.929  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 141 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7914 on 25056 degrees of freedom
  (1933 observations deleted due to missingness)
Multiple R-squared(full model): 0.4412   Adjusted R-squared: 0.4325 
Multiple R-squared(proj model): 0.1149

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”



Call:
   felm(formula = perform1 ~ experiment_treatment + treatment_post_experiment +      as.factor(year_week) | personid | 0 | personid, data = selection_data,      subset = (year_week != 201049 & balanced == 1)) 

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6041 -0.4667  0.0180  0.4926  4.0207 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.17394      0.05658   3.074  0.00251 ** 
treatment_post_experiment   0.24463      0.08832   2.770  0.00632 ** 
as.factor(year_week)201002  1.63926      0.10325  15.877  < 2e-16 ***
as.factor(year_week)201003  1.75141      0.10504  16.673  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 141 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8057 on 17918 degrees of freedom
  (1423 observations deleted due to missingness)
Multiple R-squared(full model): 0.4413   Adjusted R-squared: 0.4321 
Multiple R-squared(pro

...

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”



Call:
   felm(formula = logphonecall ~ experiment_treatment + treatment_post_experiment +      as.factor(year_week) | personid | 0 | personid, data = selection_data,      subset = (year_week != 201049 & phonecallraw > 20 & balanced ==          1)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.65587 -0.08783  0.05949  0.18796  0.93330 

Coefficients:
                           Estimate Cluster s.e. t value Pr(>|t|)    
experiment_treatment        0.09809      0.03001   3.268  0.00166 ** 
treatment_post_experiment   0.09616      0.03976   2.419  0.01810 *  
as.factor(year_week)201002  1.13169      0.07739  14.623  < 2e-16 ***
as.factor(year_week)201003  1.15556      0.07904  14.621  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 140 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3504 on 8583 degrees of freedom
  (10837 observations deleted due to missingness)
Multiple R-squared(full model): 0.2739   Adjuste

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”


feature,1,2,3,4,5,6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
experiment_treatment,0.2442*** (0.0582),0.2212*** (0.0485),0.1739*** (0.0566),0.1116*** (0.0239),0.1017*** (0.0249),0.0981*** (0.03)
treatment_post_experiment,,0.2839*** (0.0814),0.2446*** (0.0883),,0.1046*** (0.035),0.0962** (0.0398)


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Fri May 10 03:34:04 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllllll}
  \hline
 & feature & 1 & 2 & 3 & 4 & 5 & 6 \\ 
  \hline
1 & experiment_treatment & 0.2442***\\(0.0582) & 0.2212***\\(0.0485) & 0.1739***\\(0.0566) & 0.1116***\\(0.0239) & 0.1017***\\(0.0249) & 0.0981***\\(0.03) \\ 
  2 & treatment_post_experiment &  & 0.2839***\\(0.0814) & 0.2446***\\(0.0883) &  & 0.1046***\\(0.035) & 0.0962**\\(0.0398) \\ 
   \hline
\end{tabular}
\end{table}


## Table 7

In [12]:
satisfaction_data <- read_dta("data/satisfaction.dta")

col_1_regression = felm(log(satisfaction) ~ expgroup_treatment + factor(surveyno) | personid | 0 | personid, 
                        data = satisfaction_data)


summary(col_1_regression)

col_2_regression = felm(log(general) ~ expgroup_treatment + factor(surveyno) | personid | 0 | personid, 
                        data = satisfaction_data)


summary(col_2_regression)

col_3_regression = felm(log(life) ~ expgroup_treatment + factor(surveyno) | personid | 0 | personid, 
                        data = satisfaction_data)

summary(col_3_regression)

exhaustion_data <- read_dta("data/exhaustion.dta")
col_4_regression = felm(lnexhaustion ~ experiment_expgroup + announcement_expgroup + factor(year_week) | 0 | 0 | personid, 
                        data = exhaustion_data)

summary(col_4_regression)

positive_data <- read_dta("data/positive.dta")
col_5_regression = felm(lnpositive ~ experiment_expgroup + announcement_expgroup + factor(year_week) | 0 | 0 | personid, 
                        data = positive_data)

summary(col_5_regression)

negative_data <- read_dta("data/negative.dta")
col_6_regression = felm(lnnegative ~ experiment_expgroup + announcement_expgroup + factor(year_week) | 0 | 0 | personid, 
                        data = negative_data)

summary(col_6_regression)

extract_7_1_coefs <- extract_2_coefs_factory("expgroup_treatment", "announcement_expgroup")

model_list_1 <- list(col_1_regression, col_2_regression, col_3_regression)

model_data <- model_list_1 %>%
  map_dfr(extract_7_1_coefs, .id = "model")

table_data_1 <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

extract_7_2_coefs <- extract_2_coefs_factory("experiment_expgroup", "announcement_expgroup")

model_list_2 <- list(col_4_regression, col_5_regression, col_6_regression)

model_data <- model_list_2 %>%
  map_dfr(extract_7_2_coefs, .id = "model")

table_data_2 <- model_data %>%
  pivot_wider(names_from = model, values_from = values)

table_data <- cbind(table_data_1, table_data_2[, -1])
table_data
print_latex(table_data)


Call:
   felm(formula = log(satisfaction) ~ expgroup_treatment + factor(surveyno) |      personid | 0 | personid, data = satisfaction_data) 

Residuals:
     Min       1Q   Median       3Q      Max 
-1.48351 -0.09628  0.01408  0.14160  1.07536 

Coefficients:
                   Estimate Cluster s.e. t value Pr(>|t|)   
expgroup_treatment  0.15513      0.05216   2.974  0.00336 **
factor(surveyno)2  -0.05354      0.04544  -1.178  0.24035   
factor(surveyno)3  -0.10321      0.04557  -2.265  0.02477 * 
factor(surveyno)4  -0.08114      0.04413  -1.838  0.06774 . 
 [ reached getOption("max.print") -- omitted 1 row ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3193 on 679 degrees of freedom
Multiple R-squared(full model): 0.4219   Adjusted R-squared: 0.2729 
Multiple R-squared(proj model): 0.01475   Adjusted R-squared: -0.2392 
F-statistic(full model, *iid*):2.831 on 175 and 679 DF, p-value: < 2.2e-16 
F-statistic(proj model): 2.553 on 5 and


Call:
   felm(formula = log(general) ~ expgroup_treatment + factor(surveyno) |      personid | 0 | personid, data = satisfaction_data) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.61499 -0.04784  0.00277  0.05362  0.58372 

Coefficients:
                   Estimate Cluster s.e. t value Pr(>|t|)    
expgroup_treatment  0.07187      0.02125   3.382 0.000893 ***
factor(surveyno)2  -0.07991      0.01946  -4.106 6.24e-05 ***
factor(surveyno)3  -0.06754      0.01715  -3.937 0.000120 ***
factor(surveyno)4  -0.08158      0.01748  -4.666 6.20e-06 ***
 [ reached getOption("max.print") -- omitted 1 row ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1139 on 679 degrees of freedom
Multiple R-squared(full model): 0.6303   Adjusted R-squared: 0.535 
Multiple R-squared(proj model): 0.04615   Adjusted R-squared: -0.1997 
F-statistic(full model, *iid*):6.614 on 175 and 679 DF, p-value: < 2.2e-16 
F-statistic(proj model): 7.496 on 5 and 


Call:
   felm(formula = log(life) ~ expgroup_treatment + factor(surveyno) |      personid | 0 | personid, data = satisfaction_data) 

Residuals:
     Min       1Q   Median       3Q      Max 
-1.89367 -0.10466  0.00749  0.12750  0.85739 

Coefficients:
                   Estimate Cluster s.e. t value Pr(>|t|)    
expgroup_treatment  0.16814      0.04709   3.571 0.000463 ***
factor(surveyno)2  -0.07238      0.04318  -1.676 0.095519 .  
factor(surveyno)3  -0.09423      0.03759  -2.507 0.013122 *  
factor(surveyno)4  -0.11525      0.03995  -2.885 0.004421 ** 
 [ reached getOption("max.print") -- omitted 1 row ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2672 on 679 degrees of freedom
Multiple R-squared(full model): 0.6936   Adjusted R-squared: 0.6147 
Multiple R-squared(proj model): 0.02282   Adjusted R-squared: -0.229 
F-statistic(full model, *iid*):8.784 on 175 and 679 DF, p-value: < 2.2e-16 
F-statistic(proj model): 3.621 on 5 and 170


Call:
   felm(formula = lnexhaustion ~ experiment_expgroup + announcement_expgroup +      factor(year_week) | 0 | 0 | personid, data = exhaustion_data) 

Residuals:
    Min      1Q  Median      3Q     Max 
-2.1597 -1.3742  0.3881  0.8641  2.1757 

Coefficients:
                        Estimate Cluster s.e. t value Pr(>|t|)    
(Intercept)               1.7193       0.1329  12.938  < 2e-16 ***
experiment_expgroup      -0.5645       0.1680  -3.360 0.000786 ***
announcement_expgroup    -0.1022       0.1673  -0.611 0.541367    
factor(year_week)201050   0.3558       0.1093   3.254 0.001145 ** 
 [ reached getOption("max.print") -- omitted 37 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.116 on 5068 degrees of freedom
Multiple R-squared(full model): 0.06122   Adjusted R-squared: 0.05381 
Multiple R-squared(proj model): 0.06122   Adjusted R-squared: 0.05381 
F-statistic(full model, *iid*):8.263 on 40 and 5068 DF, p-value: < 2.2e-16 
F-s


Call:
   felm(formula = lnpositive ~ experiment_expgroup + announcement_expgroup +      factor(year_week) | 0 | 0 | personid, data = positive_data) 

Residuals:
     Min       1Q   Median       3Q      Max 
-1.20381 -0.12393  0.04743  0.19033  0.60429 

Coefficients:
                        Estimate Cluster s.e. t value Pr(>|t|)    
(Intercept)              3.21489      0.03490  92.106  < 2e-16 ***
experiment_expgroup      0.15970      0.04016   3.977 7.09e-05 ***
announcement_expgroup    0.08015      0.04224   1.898 0.057787 .  
factor(year_week)201050 -0.07973      0.03201  -2.491 0.012785 *  
 [ reached getOption("max.print") -- omitted 37 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2991 on 5068 degrees of freedom
Multiple R-squared(full model): 0.07379   Adjusted R-squared: 0.06648 
Multiple R-squared(proj model): 0.07379   Adjusted R-squared: 0.06648 
F-statistic(full model, *iid*):10.09 on 40 and 5068 DF, p-value: < 2.2e-


Call:
   felm(formula = lnnegative ~ experiment_expgroup + announcement_expgroup +      factor(year_week) | 0 | 0 | personid, data = negative_data) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.81996 -0.31630  0.07544  0.30110  1.07025 

Coefficients:
                        Estimate Cluster s.e. t value Pr(>|t|)    
(Intercept)              2.75030      0.04416  62.283  < 2e-16 ***
experiment_expgroup     -0.18258      0.05787  -3.155  0.00162 ** 
announcement_expgroup   -0.09495      0.05792  -1.639  0.10121    
factor(year_week)201050  0.07252      0.04299   1.687  0.09170 .  
 [ reached getOption("max.print") -- omitted 37 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3966 on 5068 degrees of freedom
Multiple R-squared(full model): 0.05137   Adjusted R-squared: 0.04388 
Multiple R-squared(proj model): 0.05137   Adjusted R-squared: 0.04388 
F-statistic(full model, *iid*):6.861 on 40 and 5068 DF, p-value: < 2.2e-

feature,1,2,3,1,2,3
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
expgroup_treatment,0.1551*** (0.0522),0.0719*** (0.0212),0.1681*** (0.0471),-0.5645*** (0.168),0.1597*** (0.0402),-0.1826*** (0.0579)
announcement_expgroup,,,,-0.1022 (0.1673),0.0802* (0.0422),-0.0949 (0.0579)


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Fri May 10 03:34:04 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllllll}
  \hline
 & feature & 1 & 2 & 3 & 1 & 2 & 3 \\ 
  \hline
1 & expgroup_treatment & 0.1551***\\(0.0522) & 0.0719***\\(0.0212) & 0.1681***\\(0.0471) & -0.5645***\\(0.168) & 0.1597***\\(0.0402) & -0.1826***\\(0.0579) \\ 
  2 & announcement_expgroup &  &  &  & -0.1022 \\(0.1673) & 0.0802*\\(0.0422) & -0.0949 \\(0.0579) \\ 
   \hline
\end{tabular}
\end{table}


## Appendix A2

In [13]:
# Load the tc_comparison_data
tc_comparison_data <- read_dta("data/tc_comparison.dta")

# Means by group
tc_comparison_data %>%
  group_by(expgroup) %>%
  summarise_all(mean, na.rm = TRUE)

# Overall SD
sapply(tc_comparison_data, sd, na.rm = TRUE)

# Significance of difference
cors <- cor(tc_comparison_data[ , c("expgroup", "age", "men", "second_technical", "high_school",
                      "tertiary_technical", "university", "prior_experience", "tenure", "married", "children",
                      "ageyoungestchild", "rental", "costofcommute", "internet", "bedroom",
                      "basewage", "bonus", "grosswage")])
corr.sig <- cor.mtest(cors, conf.level = 0.95)

# Significance of difference within type
tc_comparison_data$share <- tc_comparison_data$expgroup - 0.5
tc_comparison_data$type[tc_comparison_data$type == 3] <- 2
tc_comparison_data$type <- factor(tc_comparison_data$type)

model <- lm(share ~ type, data = tc_comparison_data)
print(coeftest(model, vcov = vcovHC(model, type = "HC0")))

expgroup,personid,prior_experience,age,tenure,basewage,bonus,grosswage,ageyoungestchild,costofcommute,⋯,rental,bedroom,men,married,second_technical,high_school,tertiary_technical,university,internet,type
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,28865.02,16.75330,24.34746,28.25424,1562.799,1092.587,3003.362,0.7118644,8.337827,⋯,0.2033898,0.9915254,0.4661017,0.3220339,0.4745763,0.1355932,0.3559322,0.03389831,1.0000000,1.906780
1,30299.21,18.96183,24.43511,26.13740,1539.864,1030.902,2949.730,0.5267176,7.891742,⋯,0.2442748,0.9694656,0.4656489,0.2213740,0.4580153,0.1755725,0.3511450,0.01526718,0.9923664,2.053435


personid   prior_experience                age             tenure 
      1.184494e+04       2.588153e+01       3.545459e+00       2.192174e+01 
          basewage              bonus          grosswage   ageyoungestchild 
      1.614480e+02       6.253335e+02       7.896269e+02       1.929177e+00 
     costofcommute           children             rental            bedroom 
      6.958090e+00       3.787410e-01       4.183571e-01       1.405576e-01 
          expgroup                men            married   second_technical 
      5.003238e-01       4.998380e-01       4.443729e-01       4.998380e-01 
       high_school tertiary_technical         university           internet 
      3.641803e-01       4.789926e-01       1.536573e-01       6.337243e-02 
              type 
      1.273071e+00


t test of coefficients:

             Estimate Std. Error t value Pr(>|t|)
(Intercept) 0.0074627  0.0431886  0.1728   0.8630
type2       0.0219491  0.0743573  0.2952   0.7681
type4       0.0203151  0.0937458  0.2167   0.8286
type5       0.2198100  0.1410561  1.5583   0.1204



In [14]:
# tc_comparison_data_ones = subset(tc_comparison_data, type == 1)
# data %>%
#   group_by(expgroup) %>%
#   summarise_all(mean, na.rm = TRUE)

# sapply(data, sd, na.rm = TRUE)

# cors <- cor(data[ , c("expgroup", "age", "men", "second_technical", "high_school",
#                       "tertiary_technical", "university", "prior_experience", "tenure", "married", "children",
#                       "ageyoungestchild", "rental", "costofcommute", "internet", "bedroom",
#                       "basewage", "bonus", "grosswage")])
# corr.sig <- cor.mtest(cors, conf.level = 0.95)

## Appendix O1

In [15]:
data <- read_dta("data/summary_volunteer.dta")
variables <- c("children", "married", "commute", "bedroom", "high_educ", "tenure", "grosswage", "age", "men")

A <- matrix(0, nrow = length(variables), ncol = 3)

j <- 1

for (i in variables) {
  # volunteer == 1
  vol1 <- data[data$volunteer == 1, ][[i]]
  A[j, 1] <- paste0(
    round(mean(vol1, na.rm = TRUE), digits = 3), 
    "\n(", 
    round(sd(vol1, na.rm = TRUE), digits = 3), 
    ")"
  )
  
  # volunteer == 0
  vol0 <- data[data$volunteer == 0, ][[i]]
  A[j, 2] <- paste0(
    round(mean(vol0, na.rm = TRUE), digits = 3), 
    "\n(", 
    round(sd(vol0, na.rm = TRUE), digits = 3), 
    ")"
  )
  
  model <- lm(data[[i]] ~ data$volunteer)
  coefs <- coef(summary(model))
  stars <- symnum(coefs[2, 4], corr = FALSE, cutpoints = star_points, symbols = star_symbols)
  A[j, 3] <- paste0(round(coefs[2, 3], digits = 3), stars)  # t-statistic
  
  j <- j + 1
}

df <- as.data.frame(A)
rownames(df) <- variables

df
print_latex(df)

,V1,V2,V3
,<chr>,<chr>,<chr>
children,0.103 (0.305),0.065 (0.247),2.168**
married,0.175 (0.38),0.126 (0.332),2.146**
commute,86.497 (60.467),74.466 (51.321),3.378***
bedroom,0.664 (0.473),0.544 (0.499),3.902***
high_educ,0.374 (0.484),0.462 (0.499),-2.839***
tenure,23.219 (20.282),26.842 (21.532),-2.732***
grosswage,2.829 (0.923),2.895 (0.944),-1.127
age,23.217 (3.284),23.23 (2.892),-0.068
men,0.342 (0.475),0.303 (0.46),1.297


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Fri May 10 03:34:05 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlll}
  \hline
 & V1 & V2 & V3 \\ 
  \hline
children & 0.103\\(0.305) & 0.065\\(0.247) & 2.168** \\ 
  married & 0.175\\(0.38) & 0.126\\(0.332) & 2.146** \\ 
  commute & 86.497\\(60.467) & 74.466\\(51.321) & 3.378*** \\ 
  bedroom & 0.664\\(0.473) & 0.544\\(0.499) & 3.902*** \\ 
  high_educ & 0.374\\(0.484) & 0.462\\(0.499) & -2.839*** \\ 
  tenure & 23.219\\(20.282) & 26.842\\(21.532) & -2.732*** \\ 
  grosswage & 2.829\\(0.923) & 2.895\\(0.944) & -1.127  \\ 
  age & 23.217\\(3.284) & 23.23\\(2.892) & -0.068  \\ 
  men & 0.342\\(0.475) & 0.303\\(0.46) & 1.297  \\ 
   \hline
\end{tabular}
\end{table}


## Appendix O4

In [16]:
# Load the treatment_effect_data
treatment_effect_data <- read_dta("data/treatment_effect.dta")
treatment_effect_data$personid <- as.character(treatment_effect_data$personid)

# List of variables
vars <- c("children", "women", "childrenwomen", "commute120", "rental", "young",
          "short_exper", "short_tenure", "livewparents", "livewspouse",
          "livewfriend", "perform10")

for (var in vars) {
  # Create interaction terms
  treatment_effect_data[, paste0("exper_treat_", var)] <- treatment_effect_data[, paste0("experiment_treatment")] * treatment_effect_data[, var]
  
  # Fit the regression model
  formula <- as.formula(paste0("perform1 ~ exper_treat_", var, " + experiment_treatment + ", var, " + factor(year_week)"))
  model <- lm(formula = formula,
              data = subset(treatment_effect_data, year_week != 201049))
  
  # Print the model summary with clustered standard errors
  print(coeftest(model, vcov = vcovCL, cluster = ~personid))
}


t test of coefficients:

                         Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)             -1.674355   0.056097 -29.8477 < 2.2e-16 ***
exper_treat_children    -0.124530   0.165913  -0.7506  0.452921    
experiment_treatment     0.238633   0.086826   2.7484  0.005995 ** 
children                 0.302966   0.095935   3.1580  0.001591 ** 
 [ reached getOption("max.print") -- omitted 84 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


t test of coefficients:

                         Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)             -1.802278   0.065962 -27.3231 < 2.2e-16 ***
exper_treat_women        0.031668   0.112389   0.2818   0.77812    
experiment_treatment     0.173105   0.097697   1.7719   0.07644 .  
women                    0.338961   0.068677   4.9356 8.064e-07 ***
 [ reached getOption("max.print") -- omitted 84 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


t test of coefficient

## Appendix O5

In [17]:
wage_new_data = read_dta("data/wage_new.dta")
performance_during_exper_copy = data.frame(performance_during_exper)
performance_during_exper_copy = performance_during_exper_copy[!is.na(performance_during_exper_copy$experiment_home) & !is.na(performance_during_exper_copy$personid) & !is.na(performance_during_exper_copy$wage_month), ]
performance_during_exper_copy$experiment_home <- suppressWarnings(ave(performance_during_exper_copy$experiment_home, 
                                              performance_during_exper_copy$personid, 
                                              performance_during_exper_copy$wage_month, 
                                              FUN = function(x) max(x, na.rm = TRUE)))
wage_data <- performance_during_exper_copy[!duplicated(performance_during_exper_copy[, c("personid", "wage_month", "experiment_treatment")]), ]
perf_wage_data <- merge(wage_data, wage_new_data, by = c("personid", "wage_month", "experiment_treatment"), all = FALSE)
head(perf_wage_data)

,personid,wage_month,experiment_treatment,year_week,expgroup,perform1,phonecall,phonecallraw,treatment,experiment_control,⋯,logcall_dayworked,logdaysworked,experiment_treatment_commute120,date,basewage,grosswage,bonustotal,loggrosswage,logbasewage,logbonustotal
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10098,201001,0,201001,0,-2.51363659,-2.42846537,89,0,0,⋯,9.761694,0.000000,0,2010-01-01,2300,4667.59,2367.59,8.448399,7.740664,7.769628
2,10098,201002,0,201006,0,-0.70797312,-0.77005094,325,0,0,⋯,9.578727,1.386294,0,2010-02-01,2300,4112.37,1825.59,8.321754,7.740664,7.509658
3,10098,201003,0,201010,0,-2.27979994,-2.39332938,94,0,0,⋯,9.720105,0.000000,0,2010-03-01,2300,3596.34,1336.00,8.187672,7.740664,7.197435
4,10098,201004,0,201015,0,0.08211296,0.16456400,458,0,0,⋯,9.624025,1.609438,0,2010-04-05,2300,4641.47,2362.62,8.442786,7.740664,7.767527
5,10098,201005,0,201019,0,-0.08289386,-0.06030576,426,0,0,⋯,9.774474,1.386294,0,2010-05-03,2300,4050.34,1790.00,8.306556,7.740664,7.489971
6,10098,201006,0,201024,0,0.76636988,0.63538504,525,0,0,⋯,9.762534,1.609438,0,2010-06-07,2300,4170.85,1892.00,8.335876,7.740664,7.545390


In [18]:
col_1_first <- felm(experiment_home ~ experiment_treatment + factor(year_week) + factor(personid) | 0 | 0 | personid,
                    data = performance_during_exper %>% filter(year_week != 201049, expgroup %in% c(1, 0)))
fitted_values <- col_1_first$fitted.values
col_1_second <- felm(perform1 ~ fitted_values + factor(year_week) + factor(personid) | 0 | 0 | personid,
                     data = performance_during_exper %>% filter(year_week != 201049, expgroup %in% c(1, 0))) 

summary(col_1_first)
summary(col_1_second)

col_2_first <- felm(experiment_home ~ experiment_treatment + factor(wage_month) + factor(personid) | 0 | 0 | personid, 
                    data = perf_wage_data %>% filter(wage_month < 201109))
fitted_values <- col_2_first$fitted.values
col_2_second <- felm(loggrosswage ~ fitted_values + factor(wage_month) + factor(personid) | 0 | 0 | personid, 
                     data = perf_wage_data %>% filter(wage_month < 201109))

summary(col_2_first)
summary(col_2_second)

col_3_first <- felm(homethatweek ~ treatment + factor(year_week) | 0 | 0 | personid, 
                    data = perf_wage_data %>% filter(year_week > 201049, expgroup %in% c(1, 0)))
fitted_values <- col_3_first$fitted.values
col_3_second <- felm(perform1 ~ fitted_values + factor(year_week) | 0 | 0 | personid, 
                     data = perf_wage_data %>% filter(year_week > 201049, expgroup %in% c(1, 0)))

summary(col_3_first)
summary(col_3_second)

col_4_first <- felm(experiment_home ~ experiment_treatment + factor(year_week) + factor(personid) | 0 | 0 | personid,
                    data = perf_wage_data %>% filter(year_week != 201049, expgroup %in% c(1, 0), !is.na(logcallpersec), phonecallraw > 20))
fitted_values <- col_4_first$fitted.values
col_4_second <- felm(phonecall ~ fitted_values + factor(year_week) + factor(personid) | 0 | 0 | personid,
                     data = perf_wage_data %>% filter(year_week != 201049, expgroup %in% c(1, 0), !is.na(logcallpersec), phonecallraw > 20))

summary(col_4_first)
summary(col_4_second)

col_5_first <- col_4_first
col_6_first <- col_4_first
col_7_first <- col_4_first

col_5_second <- felm(logphonecall ~ fitted_values + factor(year_week) + factor(personid) | 0 | 0 | personid,
                     data = perf_wage_data %>% filter(year_week != 201049, expgroup %in% c(1, 0), !is.na(logcallpersec), phonecallraw > 20))

summary(col_5_second)

col_6_second <- felm(logcallpersec ~ fitted_values + factor(year_week) + factor(personid) | 0 | 0 | personid,
                     data = perf_wage_data %>% filter(year_week != 201049, expgroup %in% c(1, 0), !is.na(logcallpersec), phonecallraw > 20))

summary(col_6_second)

col_7_second <- felm(logcalllength ~ fitted_values + factor(year_week) + factor(personid) | 0 | 0 | personid,
                     data = perf_wage_data %>% filter(year_week != 201049, expgroup %in% c(1, 0), !is.na(logcallpersec), phonecallraw > 20))

summary(col_7_second)

model_list_1 <- list(col_1_first, col_3_first, col_4_first, col_5_first, col_6_first, col_7_first, col_2_first)
model_data <- model_list_1 %>% map_dfr(extract_first_regressor, .id = "model")
table_data_1 <- model_data %>% pivot_wider(names_from = model, values_from = values)

model_list_2 <- list(col_1_second, col_3_second, col_4_second, col_5_second, col_6_second, col_7_second, col_2_second)
model_data <- model_list_2 %>% map_dfr(extract_first_regressor, .id = "model")
table_data_2 <- model_data %>% pivot_wider(names_from = model, values_from = values)

table_data <- rbind(table_data_2, table_data_1)
table_data
print_latex(table_data)

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”



Call:
   felm(formula = experiment_home ~ experiment_treatment + factor(year_week) +      factor(personid) | 0 | 0 | personid, data = performance_during_exper %>%      filter(year_week != 201049, expgroup %in% c(1, 0))) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.92928 -0.02929  0.00314  0.04318  0.99073 

Coefficients:
                          Estimate Cluster s.e.  t value Pr(>|t|)    
(Intercept)             -3.096e-03    1.377e-02   -0.225 0.822150    
experiment_treatment     8.565e-01    2.333e-02   36.708  < 2e-16 ***
factor(year_week)201002  5.323e-04    1.675e-03    0.318 0.750657    
factor(year_week)201003  1.377e-03    1.744e-03    0.789 0.429922    
 [ reached getOption("max.print") -- omitted 330 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1484 on 18177 degrees of freedom
Multiple R-squared(full model): 0.8646   Adjusted R-squared: 0.8621 
Multiple R-squared(proj model): 0.8646   Adjusted R-squa

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”



Call:
   felm(formula = perform1 ~ fitted_values + factor(year_week) +      factor(personid) | 0 | 0 | personid, data = performance_during_exper %>%      filter(year_week != 201049, expgroup %in% c(1, 0))) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.2689 -0.4187  0.0424  0.4687  3.7219 

Coefficients:
                         Estimate Cluster s.e.  t value Pr(>|t|)    
(Intercept)             -1.524771     0.065288  -23.354  < 2e-16 ***
fitted_values            0.271301     0.073315    3.700 0.000216 ***
factor(year_week)201002  1.629271     0.085790   18.991  < 2e-16 ***
factor(year_week)201003  1.769144     0.089280   19.816  < 2e-16 ***
 [ reached getOption("max.print") -- omitted 330 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7826 on 17472 degrees of freedom
  (705 observations deleted due to missingness)
Multiple R-squared(full model): 0.4018   Adjusted R-squared: 0.3904 
Multiple R-squared(proj model): 0.401

Warning message in sqrt(diag(z$STATS[[lhs]]$clustervcv)):
“NaNs produced”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”


...

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”



Call:
   felm(formula = logcalllength ~ fitted_values + factor(year_week) +      factor(personid) | 0 | 0 | personid, data = perf_wage_data %>%      filter(year_week != 201049, expgroup %in% c(1, 0), !is.na(logcallpersec),          phonecallraw > 20)) 

Residuals:
     Min       1Q   Median       3Q      Max 
-2.77945 -0.10463  0.05319  0.19169  0.95561 

Coefficients:
                          Estimate Cluster s.e.    t value Pr(>|t|)    
(Intercept)              1.029e+01    5.955e-02  1.728e+02  < 2e-16 ***
fitted_values            1.057e-01    4.591e-02  2.302e+00 0.021432 *  
factor(year_week)201002  1.169e+00    1.941e-01  6.022e+00 2.03e-09 ***
factor(year_week)201005 -4.230e-01    1.629e-01 -2.596e+00 0.009502 ** 
 [ reached getOption("max.print") -- omitted 184 rows ]
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.369 on 2115 degrees of freedom
Multiple R-squared(full model): 0.4871   Adjusted R-squared: 0.4418 
Multiple R-squar

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive de

regressor,1,2,3,4,5,6,7
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
fitted_values,0.2713*** (0.0733),0.2032* (0.1052),0.2783*** (0.0901),0.1424*** (0.0451),0.0367** (0.0155),0.1057** (0.0459),0.0771** (0.0313)
experiment_treatment,0.8565*** (0.0233),NA,0.8983*** (0.0307),0.8983*** (0.0307),0.8983*** (0.0307),0.8983*** (0.0307),0.8808*** (0.0229)
treatment,NA,0.8543*** (0.0229),NA,NA,NA,NA,NA


% latex table generated in R 4.4.0 by xtable 1.8-4 package
% Fri May 10 03:34:17 2024
\begin{table}[ht]
\centering
\begin{tabular}{rllllllll}
  \hline
 & regressor & 1 & 2 & 3 & 4 & 5 & 6 & 7 \\ 
  \hline
1 & fitted_values & 0.2713***\\(0.0733) & 0.2032*\\(0.1052) & 0.2783***\\(0.0901) & 0.1424***\\(0.0451) & 0.0367**\\(0.0155) & 0.1057**\\(0.0459) & 0.0771**\\(0.0313) \\ 
  2 & experiment_treatment & 0.8565***\\(0.0233) &  & 0.8983***\\(0.0307) & 0.8983***\\(0.0307) & 0.8983***\\(0.0307) & 0.8983***\\(0.0307) & 0.8808***\\(0.0229) \\ 
  3 & treatment &  & 0.8543***\\(0.0229) &  &  &  &  &  \\ 
   \hline
\end{tabular}
\end{table}
